In [ ]:
!pip install -q ../input/segmentation-models-wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl 
!pip install -q ../input/segmentation-models-wheels/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install -q ../input/segmentation-models-wheels/timm-0.3.2-py3-none-any.whl
!pip install -q ../input/segmentation-models-wheels/segmentation_models_pytorch-0.1.3-py3-none-any.whl

In [ ]:
import wandb
import os
import cv2
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
import torch
import collections.abc as container_abcs
torch._six.container_abcs = container_abcs
import segmentation_models_pytorch as smp
warnings.filterwarnings("ignore")


def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
fix_all_seeds(2021)

# from wandb.keras import WandbCallback

In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
class config:
    seed          = 101
    debug         = False # set debug=False for Full Training
    exp_name      = 'Unet-effnetb2-512x512-aug2'
    model_name    = 'Unet'
    backbone0     = 'efficientnet-b2'
    backbone1     = 'efficientnet-b7'
    backbone2     = 'efficientnet-b6'
    backbone3     = 'efficientnet-b5'
    batch_size    = 8
    train_bs      = 24
    valid_bs      = 48
    IMAGE_RESIZE  = (512, 512)
    epochs        = 2
    lr            = 5e-3
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(100*6*1.8)
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = 32//train_bs
#     n_fold        = 5
    num_classes   = 1
    DEVICE        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    competition   = 'sartorius'
    _wandb_kernel = 'somusan'
    TEST_PATH     = "../input/sartorius-cell-instance-segmentation/test"

In [ ]:
model = smp.Unet(config.backbone0, encoder_weights="imagenet", classes=1,activation=None)
model1 = smp.Unet(config.backbone1, encoder_weights="imagenet", classes=1,activation=None)
model2 = smp.Unet(config.backbone2, encoder_weights="imagenet", classes=1,activation=None)
model3 = smp.Unet(config.backbone3, encoder_weights="imagenet", classes=1,activation=None)

In [ ]:
best_model0 = wandb.restore('model0.pth', run_path="somusan/kaggle_sartorius_unet_ensemble/dnhtjlrc")
model.load_state_dict(torch.load(best_model0.name))

best_model1 = wandb.restore('model1.pth', run_path="somusan/kaggle_sartorius_unet_ensemble/dnhtjlrc")
model1.load_state_dict(torch.load(best_model1.name))

best_model2 = wandb.restore('model2.pth', run_path="somusan/kaggle_sartorius_unet_ensemble/dnhtjlrc")
model2.load_state_dict(torch.load(best_model2.name))

best_model3 = wandb.restore('model3.pth', run_path="somusan/kaggle_sartorius_unet_ensemble/dnhtjlrc")
model3.load_state_dict(torch.load(best_model3.name))

In [ ]:
class TestCellDataset(Dataset):
    def __init__(self):
        self.test_path = config.TEST_PATH
        
        
        self.image_ids = [f[:-4]for f in os.listdir(self.test_path)]
        self.num_samples = len(self.image_ids)
        self.transform = A.Compose([A.Resize(config.IMAGE_RESIZE[0], config.IMAGE_RESIZE[1]), ToTensorV2()])

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        path = os.path.join(self.test_path, image_id + ".png")
        image = cv2.imread(path)
        image = self.transform(image=image)['image']
        return {'image': image, 'id': image_id}

    def __len__(self):
        return self.num_samples

In [ ]:
ds_test = TestCellDataset()
dl_test = DataLoader(ds_test, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
def post_process(probability, threshold=0.5, min_size=300):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = []
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            a_prediction = np.zeros((520, 704), np.float32)
            a_prediction[p] = 1
            predictions.append(a_prediction)
    return predictions


def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

In [ ]:
def get_preds(model):
    model.eval()
    submission = []
    for i, batch in enumerate(tqdm(dl_test)):
        preds = torch.sigmoid(model(batch['image'].float()))
        preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
        for image_id, probability_mask in zip(batch['id'], preds):
            probability_mask = cv2.resize(probability_mask, dsize=(704, 520), interpolation=cv2.INTER_LINEAR)
            submission.append(probability_mask)
    return submission

In [ ]:
plt.imshow(submission[0]>0.5)

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
# img = cv.imread('opencv_logo.png')
plt.figure(figsize=(20,20))
img = submission[0] > 0.5
# kernel = np.ones((5,5),np.float32)/25
# dst = cv.filter2D(submission[0],-1,kernel)
dst = cv2.medianBlur(img, 5)
plt.subplot(121),plt.imshow(img),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(dst),plt.title('Averaging')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
model.eval()

submission = []
for i, batch in enumerate(tqdm(dl_test)):
    preds = torch.sigmoid(model(batch['image'].cuda()))
    preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    for image_id, probability_mask in zip(batch['id'], preds):
        try:
            #if probability_mask.shape != IMAGE_RESIZE:
            #    probability_mask = cv2.resize(probability_mask, dsize=IMAGE_RESIZE, interpolation=cv2.INTER_LINEAR)
            probability_mask = cv2.resize(probability_mask, dsize=(704, 520), interpolation=cv2.INTER_LINEAR)
            predictions = post_process(probability_mask)
            for prediction in predictions:
                #plt.imshow(prediction)
                #plt.show()
                try:
                    submission.append((image_id, rle_encoding(prediction)))
                except:
                    print("Error in RL encoding")
        except Exception as e:
            print(f"Exception for img: {image_id}: {e}")
        
        # Fill images with no predictions
        image_ids = [image_id for image_id, preds in submission]
        if image_id not in image_ids:
            submission.append((image_id, ""))
            
df_submission = pd.DataFrame(submission, columns=['id', 'predicted'])
df_submission.to_csv('submission.csv', index=False)

if df_submission['predicted'].apply(check_is_run_length).mean() != 1:
    print("Check run lenght failed")
    create_empty_submission()

In [ ]:

# from keras.models import load_model

#Set compile=False as we are not loading it for training, only for prediction.
# model1 = load_model('saved_models/res34_backbone_50epochs.hdf5', compile=False)
# model2 = load_model('saved_models/inceptionv3_backbone_50epochs.hdf5', compile=False)
# model3 = load_model('saved_models/vgg19_backbone_50epochs.hdf5', compile=False)

#Weighted average ensemble
# models = [model1, model2, model3]
#preds = [model.predict(X_test) for model in models]

# pred1 = model1.predict(X_test1)
# pred2 = model2.predict(X_test2)
# pred3 = model3.predict(X_test3)

pred0 = get_preds(model)
pred1 = get_preds(model1)
pred2 = get_preds(model2)
pred3 = get_preds(model3)



preds=np.array([pred0[0], pred1[0], pred2[0], pred3[0]])

#preds=np.array(preds)
weights = [0.3, 0.5, 0.3,0.4]

#Use tensordot to sum the products of all elements over specified axes.
weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
weighted_ensemble_prediction = np.argmax(weighted_preds, axis=3)

y_pred1_argmax=np.argmax(pred1, axis=3)
y_pred2_argmax=np.argmax(pred2, axis=3)
y_pred3_argmax=np.argmax(pred3, axis=3)


#Using built in keras function
n_classes = 1
IOU1 = MeanIoU(num_classes=n_classes)  
IOU2 = MeanIoU(num_classes=n_classes)  
IOU3 = MeanIoU(num_classes=n_classes)  
IOU_weighted = MeanIoU(num_classes=n_classes)  

IOU1.update_state(y_test[:,:,:,0], y_pred1_argmax)
IOU2.update_state(y_test[:,:,:,0], y_pred2_argmax)
IOU3.update_state(y_test[:,:,:,0], y_pred3_argmax)
IOU_weighted.update_state(y_test[:,:,:,0], weighted_ensemble_prediction)


print('IOU Score for model1 = ', IOU1.result().numpy())
print('IOU Score for model2 = ', IOU2.result().numpy())
print('IOU Score for model3 = ', IOU3.result().numpy())
print('IOU Score for weighted average ensemble = ', IOU_weighted.result().numpy())